## Doing spatial requests

In order to do regular spatial requests, we need the following data
 - The users location (specified in (lat,lon) format, currently set in Barcelona area)
 - The greater circle in kilometers for which we would like to limit the spatial search (two kilometers)
 - A regular user query (*:*, for example)
 - The spatial field to filter and order by (location)
 
As extra parameters, we need to tell that the Filter Query would use the **geofilt!** functionality where the greater circle is the one specified above.

Below we perform only a star-doublecolon-star query, but getting the results in 2kms distance

In [1]:
import requests

# barcelona #41.3990912,2.1657931 #_radius=30
#zaragoza #41.622393,-0.9108323 #_radius=120
_url = "http://solr:8983/solr/retrotech/select"
_params = {
    'defType': "edismax",
    'fq': "{!geofilt}",          # filter query that applies the geofilt function given a distance 'd' 
    'q.op':"OR",
    'd':"5",                     # greater circle distance in kilometers
    'pt':"41.3990912,2.1657931", # latitude and longitude of point (Barcelona city)
    'sfield': "location",         # spatial field for filtering and sorting
    'rows': "900",
    'q':"*:*",
    'qf':"name",
    'fl':"name,location,geohash",
    'debugQuery':"true"

}

url = "http://0.0.0.0:8983/solr/retrotech/select"# ?debugQuery=true&defType="+_deftype+"&fl"+_fl+"&fq="+_fq+"&indent=true&q.op=OR&q=*:*&qf=name&rows=900&d=2&pt="+_pt+"&sfield="+_sfield+"&spatial=true"

data = requests.get(url = _url, params = _params).json()


Below list should give you list of names of the product and its latitude and longitude. Please note how we got 16 results instead of 900. This is because we are only getting results in 5 kms distance.

In [2]:
print(*((x['name'],x['location']) for x in data['response']['docs']), sep='\n')

('Samsung - 64\\ Class - Plasma - 1080p - 600Hz - Smart - 3D - HDTV"', '41.3700889,2.1307976')
('Samsung - 22\\ Class / 1080p / 60Hz / LCD HDTV"', '41.3700889,2.1307976')
('Bulletproof Monk - Widescreen - Blu-ray Disc', '41.3993586,2.1976986')
('Phineas & Ferb: The Fast and the Phineas - Fullscreen - DVD', '41.3700889,2.1307976')
('Western Digital - My Passport Essential 500GB External USB 3.0/2.0 Portable Hard Drive - Real Red', '41.3700889,2.1307976')
('Tribeca - Atlanta Braves Case for Apple&#xAE; iPhone&#xAE; 4 - Dark Blue', '41.3700889,2.1307976')
('Metra - Turbokits Aftermarket Radio Wire Harness Adapter for Select Ford Vehicles', '41.3700889,2.1307976')
('Believers [LP] - VINYL', '41.4080147,2.2091094')
('I Never Promised You a Rose Garden: A Portrait of David Toop Through His Record Collection - DVD', '41.3700889,2.1307976')
('Scosche - Showtime Composite Video Cable for Apple&#xAE; iPod&#xAE; and iPhone', '41.3700889,2.1307976')
('Honeywell - Easy-to-Care 1 Gal. Cool Moisture 

### Suggestions time?

This same logic is the one used with the `suggest` component, that was extended with geospatial features, where the lookup implementation would factor in the geospatial score. This combination was achieved like that and with some custom joint-queries and subqueries.

I could not bring that customisation on this hands-on , sorry D: 


But for the sake of getting results over suggestions without the suggester component, we are doing a regular query to have distance-based results, with some tweaks as below 👇🏻 
 - Enlarging our greater circle radius **(to 30 kms)**
 - Doing a very recall-oriented autocomplete request, with the term **"DVD"**
 - With doing a "like" query using the Extended Dismax parser
 


We could boost the item titles by giving more weight to the prefixed ones, but for this example we won't do it.

In [6]:
_q="DVD"
# barcelona #41.3990912,2.1657931 #_radius=30
#zaragoza #41.622393,-0.9108323 #_radius=120
_point="41.3990912,2.1657931" # 
_url = "http://solr:8983/solr/retrotech/select"
_radius=30
#_radius=120 # zaragoza
_params = {
    'defType': "edismax",
    'fq': "{!geofilt}",          # filter query that applies the geofilt function given a distance 'd' 
    'q.op':"OR",
    'd':_radius,                     # greater circle distance in kilometers
    'pt':"41.3990912,2.1657931", # latitude and longitude of point
#   'pt':"41.622393,-0.9108323", # latitude and longitude of point in zaragoza
    'sfield': "location",         # spatial field for filtering and sorting
    'rows': "90",
    'q':"name:*"+_q+"*",          #ugly but working nonetheless
    'qf':"name",
    'fl':"name,location,geohash",
    'debugQuery':"true",
    #'facet': 'true',
    #'facet.field': 'name',
    #'f.names.facet.prefix':"$"+_p+"$"
}

url = "http://0.0.0.0:8983/solr/retrotech/select"# ?debugQuery=true&defType="+_deftype+"&fl"+_fl+"&fq="+_fq+"&indent=true&q.op=OR&q=*:*&qf=name&rows=900&d=2&pt="+_pt+"&sfield="+_sfield+"&spatial=true"

data = requests.get(url = _url, params = _params)
print(data.text)


{
  "responseHeader":{
    "status":0,
    "QTime":529,
    "params":{
      "q":"name:*DVD*",
      "defType":"edismax",
      "d":"30",
      "pt":"41.3990912,2.1657931",
      "qf":"name",
      "fl":"name,location,geohash",
      "q.op":"OR",
      "fq":"{!geofilt}",
      "sfield":"location",
      "rows":"90",
      "debugQuery":"true"}},
  "response":{"numFound":655,"start":0,"numFoundExact":true,"docs":[
      {
        "name":"Of Mice and Men - DVD",
        "location":"41.4970697,2.3214331",
        "geohash":"sp3sh"},
      {
        "name":"Night of the Living Dead/Sleepy Hollow High [2 Discs] - Box - DVD",
        "location":"41.5393484,2.2130934",
        "geohash":"sp3s6"},
      {
        "name":"Moral Orel, Vol. 1: The Unholy Version [2 Discs] - Uncensored Special - DVD",
        "location":"41.3163083,2.0156034",
        "geohash":"sp37j"},
      {
        "name":"Half Nelson - Widescreen Subtitle AC3 Dolby - DVD",
        "location":"41.5398348,2.4448926",
        "g

Let's plot them on a map now so we can see it :D 

In [8]:
from ipyleaflet import Map, basemaps, Circle
_json = data.json()

# barcelona #41.3990912,2.1657931
#zaragoza #41.622393,-0.9108323

center = [41.3990912,2.1657931]
#center = [41.622393,-0.9108323] #zaragoza
zoom = 8

m = Map(basemap=basemaps.OpenStreetMap.Mapnik, center=center, zoom=zoom)

for r in _json['response']['docs']:
    p = Circle()
    p.radius=2
    p.weight=2
    p.color="blue"
    p.fill_color="blue"
    p.location = tuple(r['location'].split(','))
    m.add(p)
m


r = Circle()
r.radius = _radius*1000
r.weight=2
r.color = "orange"
r.fill_color = "orange"
_bcn_latlon = (41.3990912,2.1657931)
#_bcn_latlon = (41.622393,-0.9108323)

r.location = _bcn_latlon
m.add(r)
m

Map(center=[41.3990912, 2.1657931], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title'…